In [ ]:
import sqlite3
from datetime import datetime, timedelta
import os

script_dir = os.getcwd()
print(script_dir)

background_data_dir = os.path.join(script_dir, 'Background Data Folder')
# check if the directory exists, if not, create it
if not os.path.exists(background_data_dir):
    os.makedirs(background_data_dir)

week_dates_path = os.path.join(background_data_dir, 'week_dates.db')
master_data_path = os.path.join(background_data_dir, 'master_data.db')

source_conn = sqlite3.connect(week_dates_path) 
source_cursor = source_conn.cursor()
source_cursor.execute("SELECT StartDate, EndDate FROM week_dates ORDER BY id ASC")
rows = source_cursor.fetchall()
source_conn.close()

transformed_data = []

for start_str, end_str in rows:
    start_date = datetime.fromisoformat(start_str)
    end_date = datetime.fromisoformat(end_str)

    week_number = int(start_date.strftime("%U")) + 1  # Week number (1-based)
    year_week = 202500 + week_number

    formatted_start = start_date.strftime("%d %b")
    formatted_end_minus_1 = (end_date - timedelta(days=1)).strftime("%d %b")
    current_sheet_name = f"{formatted_start} - {formatted_end_minus_1}"

    full_start = start_date.strftime("%d %b %Y")
    full_end = end_date.strftime("%d %b %Y")
    current_period = f"{full_start} - {full_end}"

    period_name = f"{year_week} - {current_period}"

    # Calculate ReportingDate as the Friday following EndDate
    reporting_date = (end_date + timedelta(days=1)).strftime("%d %b %Y")

    transformed_data.append({
        "StartDate": start_str,
        "EndDate": end_str,
        "WeekNumber": week_number,
        "YearWeek": year_week,
        "CurrentSheetName": current_sheet_name,
        "CurrentPeriod": current_period,
        "PeriodName": period_name,
        "ReportingDate": reporting_date
    })

# Add Previous and Next PeriodNames
for i, row in enumerate(transformed_data):
    row["PreviousSheetName"] = transformed_data[i - 1]["CurrentSheetName"] if i > 0 else None
    row["NextSheetName"] = transformed_data[i + 1]["CurrentSheetName"] if i < len(transformed_data) - 1 else None

# Connect to MasterData.db and recreate the table
master_conn = sqlite3.connect(master_data_path)
master_cursor = master_conn.cursor()

master_cursor.execute("DROP TABLE IF EXISTS MasterData")

master_cursor.execute("""
CREATE TABLE MasterData (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    StartDate TEXT,
    EndDate TEXT,
    WeekNumber INTEGER,
    YearWeek INTEGER,
    CurrentSheetName TEXT,
    CurrentPeriod TEXT,
    PeriodName TEXT,
    PreviousSheetName TEXT,
    NextSheetName TEXT,
    ReportingDate TEXT
)
""")

# Insert all data
master_cursor.executemany("""
INSERT INTO MasterData (
    StartDate, EndDate,
    WeekNumber, YearWeek,
    CurrentSheetName, CurrentPeriod,
    PeriodName, PreviousSheetName, NextSheetName,
    ReportingDate
) VALUES (
    :StartDate, :EndDate,
    :WeekNumber, :YearWeek,
    :CurrentSheetName, :CurrentPeriod,
    :PeriodName, :PreviousSheetName, :NextSheetName,
    :ReportingDate
)
""", transformed_data)

master_conn.commit()
master_conn.close()
